In [7]:
owner = 'team-18'

In [8]:
import polaris as po
import datamol as dm
import numpy as np
import warnings


In [9]:
benchmark = po.load_benchmark("polaris/pkis1-kit-wt-mut-c-1")
train, test = benchmark.get_train_test_split(featurization_fn=dm.to_fp)
#train, test = benchmark.get_train_test_split()

ys = train.y
ys = np.stack([ys[target] for target in benchmark.target_cols], axis=1)
mask = ~np.any(np.isnan(ys), axis=1)
mask.sum()

X_train = train.X[mask]
y_train = ys[mask]

print(y_train.shape)



2024-06-21 16:23:49.029 | INFO     | polaris._artifact:_validate_version:66 - The version of Polaris that was used to create the artifact (0.0.0) is different from the currently installed version of Polaris (dev).
2024-06-21 16:23:49.033 | INFO     | polaris._artifact:_validate_version:66 - The version of Polaris that was used to create the artifact (0.0.0) is different from the currently installed version of Polaris (dev).


(276, 3)


In [10]:
from functools import partial
        
from imblearn.over_sampling import SMOTE
import polaris as po
import datamol as dm
import numpy as np

mapping = {
 (0.0, 0.0, 0.0): 0,
 (1.0, 0.0, 0.0): 1,
 (1.0, 0.0, 1.0): 2,
 (1.0, 1.0, 0.0): 3,
 (1.0, 1.0, 1.0): 4,
}
inv_mapping = {v: k for k, v in mapping.items()}

# load dataset
benchmark = po.load_benchmark("polaris/pkis1-kit-wt-mut-c-1")
# use ECFP fingerprint
train, test = benchmark.get_train_test_split(featurization_fn=partial(dm.to_fp, fp_type='ecfp'))

# define order of target values
target_order = ['CLASS_KIT', 'CLASS_KIT_(T6701_mutant)', 'CLASS_KIT_(V560G_mutant)']

# reshape the y values for convenience
ys = train.y
ys = np.stack([ys[target] for target in target_order], axis=1)
ys.shape

# remove the rows with NaN values
mask = ~np.any(np.isnan(ys), axis=1)
mask.sum()
X = train.X[mask]
ys = ys[mask]

ys_scalarized = [tuple(item) for item in ys]
ys_scalarized = [mapping[item] for i, item in enumerate(ys_scalarized)]

X_train, y_resampled = SMOTE(k_neighbors=2).fit_resample(X, ys_scalarized)
y_train = [inv_mapping[item] for i, item in enumerate(y_resampled)]

y_train = np.array(y_train)


2024-06-21 16:23:50.551 | INFO     | polaris._artifact:_validate_version:66 - The version of Polaris that was used to create the artifact (0.0.0) is different from the currently installed version of Polaris (dev).
2024-06-21 16:23:50.556 | INFO     | polaris._artifact:_validate_version:66 - The version of Polaris that was used to create the artifact (0.0.0) is different from the currently installed version of Polaris (dev).


In [11]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from bayes_opt import BayesianOptimization
from scikeras.wrappers import KerasClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import make_scorer, average_precision_score

def get_param(optimizer):    
    best_params = optimizer.max['params']
    best_params['input_dim'] = int(best_params['input_dim'])
    best_params['learning_rate'] = int(best_params['learning_rate'])
    best_params['dropout_rate'] = int(best_params['dropout_rate'])
    best_params['num_neurons_layer1'] = int(best_params['num_neurons_layer1'])
    best_params['num_neurons_layer2'] = int(best_params['num_neurons_layer2'])
    return best_params

def opt_multi_targets(param_bounds):


    # benchmark = po.load_benchmark("polaris/pkis1-kit-wt-mut-c-1")
    # train, test = benchmark.get_train_test_split(featurization_fn=dm.to_fp)
    # #train, test = benchmark.get_train_test_split()
    
    # ys = train.y
    # ys = np.stack([ys[target] for target in benchmark.target_cols], axis=1)
    # mask = ~np.any(np.isnan(ys), axis=1)
    # mask.sum()
    
    # X_train = train.X[mask]
    # y_train = ys[mask]

    X_train, y_resampled = SMOTE(k_neighbors=2).fit_resample(X, ys_scalarized)
    y_train = [inv_mapping[item] for i, item in enumerate(y_resampled)]
    y_train = np.array(y_train)
  
    def create_model(input_dim, learning_rate, dropout_rate, num_neurons_layer1, num_neurons_layer2):
        model = Sequential()
        model.add(Dense(int(num_neurons_layer1), input_dim=input_dim, activation='relu', kernel_regularizer='l2'))
        model.add(Dropout(dropout_rate))
        model.add(Dense(int(num_neurons_layer2), activation='relu', kernel_regularizer='l2'))
        model.add(Dropout(dropout_rate))
        model.add(Dense(3, activation='sigmoid'))  # 3 output classes
        model.compile(loss='bce', optimizer=Adam(learning_rate=learning_rate), metrics=['accuracy'])
        return model

    def optimize_model(learning_rate, dropout_rate, num_neurons_layer1, num_neurons_layer2):
        kfold = KFold(n_splits=5, shuffle=True, random_state=42)
        accuracies = []
    
        for train_index, val_index in kfold.split(X_train):
            X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
            y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]
            
            model = create_model(learning_rate, dropout_rate, num_neurons_layer1, num_neurons_layer2)
            early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
            
            model.fit(X_train_fold, y_train_fold, epochs=100, batch_size=16, verbose=0, validation_data=(X_val_fold, y_val_fold), callbacks=[early_stopping])
            _, accuracy = model.evaluate(X_val_fold, y_val_fold, verbose=0)
            accuracies.append(accuracy)
        
        return np.mean(accuracies)

    def objective(input_dim, num_neurons_layer1, num_neurons_layer2, dropout_rate, learning_rate):
        dnn = KerasClassifier(model=create_model, 
                              input_dim = int(input_dim),
                              learning_rate=learning_rate, 
                              dropout_rate=dropout_rate, 
                              num_neurons_layer1=num_neurons_layer1, 
                              num_neurons_layer2=num_neurons_layer2,
                              epochs=50, 
                              batch_size=16, 
                              verbose=0)

        pipeline = Pipeline([
            ('pca', PCA(int(input_dim))),
            ('scaler', StandardScaler()),
            ('dnn', dnn)
        ])
        
        # Use K-Fold cross-validation
        kfold = KFold(n_splits=2, shuffle=True, random_state=42)
        
        # Evaluate the model using cross-validation
        scoring = make_scorer(average_precision_score, needs_proba=True)
        
        cv_scores = cross_val_score(pipeline, X_train, y_train, cv=kfold, scoring=scoring)
        return cv_scores.mean()

    

    optimizer = BayesianOptimization(
        f=objective,
        pbounds=param_bounds,
        verbose=2,
        random_state=42
    )
    
    optimizer.maximize(init_points=5, n_iter=15)

    best_params = optimizer.max['params']
    print("Best parameters: ", best_params)
    
    # Train the final model with the best parameters
    # final_model = create_model(best_params['learning_rate'], best_params['dropout_rate'], best_params['num_neurons_layer1'], best_params['num_neurons_layer2'])
    # early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    # final_model.fit(X_train, y_train, epochs=100, batch_size=16, verbose=2, validation_split=0.2, callbacks=[early_stopping])

    # dnn = KerasClassifier(model=create_model, 
    #                       input_dim = best_params['input_dim'],
    #                       learning_rate=best_params['learning_rate'], 
    #                       dropout_rate=best_params['dropout_rate'], 
    #                       num_neurons_layer1=best_params['num_neurons_layer1'], 
    #                       num_neurons_layer2=best_params['num_neurons_layer2'],
    #                       epochs=50, 
    #                       batch_size=16, 
    #                       verbose=0)
    
    
    #dnn.fit(X_train, y_train)

    # Evaluate the final model  
    

    # optimizer = BayesianOptimization(f=objective, pbounds=param_bounds, random_state=42, verbose = 2)
    
    # optimizer.maximize(init_points=5, n_iter=5)
    
    # best_params = get_param(optimizer)

    # best_dnn = KerasClassifier(build_fn=create_model, 
    #                            num_neurons_layer1=best_params['num_neurons_layer1'], 
    #                            num_neurons_layer2=best_params['num_neurons_layer2'], 
    #                            dropout_rate=best_params['dropout_rate'], 
    #                            learning_rate=best_params['learning_rate'],
    #                            epochs=50, 
    #                            batch_size=16, 
    #                            verbose=2)

    
    # # Evaluate the final model
    # loss, accuracy = best_pipeline.evaluate(X_test, y_test, verbose=2)
    # print(f"Final model accuracy: {accuracy}")
    
    return best_params


In [12]:
param_bounds = {
    'learning_rate': (0.0001, 0.01),
    'input_dim': (10, 100),
    'dropout_rate': (0.0, 0.3),
    'num_neurons_layer1': (10, 100),  # Layer 1 neurons
    'num_neurons_layer2': (10, 100)   # Layer 2 neurons
}

best_params = opt_multi_targets(param_bounds = param_bounds)

|   iter    |  target   | dropou... | input_dim | learni... | num_ne... | num_ne... |
-------------------------------------------------------------------------------------


/home/ubuntu/micromamba/envs/moml-ilya/lib/python3.11/site-packages/sklearn/metrics/_scorer.py:610: FutureWarning: The `needs_threshold` and `needs_proba` parameter are deprecated in version 1.4 and will be removed in 1.6. You can either let `response_method` be `None` or set it to `predict` to preserve the same behaviour.
  warnings.warn(
/home/ubuntu/micromamba/envs/moml-ilya/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/home/ubuntu/micromamba/envs/moml-ilya/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__

| 1         | 0.9982    | 0.1124    | 95.56     | 0.007347  | 63.88     | 24.04     |


/home/ubuntu/micromamba/envs/moml-ilya/lib/python3.11/site-packages/sklearn/metrics/_scorer.py:610: FutureWarning: The `needs_threshold` and `needs_proba` parameter are deprecated in version 1.4 and will be removed in 1.6. You can either let `response_method` be `None` or set it to `predict` to preserve the same behaviour.
  warnings.warn(
/home/ubuntu/micromamba/envs/moml-ilya/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/home/ubuntu/micromamba/envs/moml-ilya/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__

| 2         | 0.9927    | 0.0468    | 15.23     | 0.008675  | 64.1      | 73.73     |


...

| 18        | 0.9973    | 0.1612    | 86.28     | 0.008601  | 62.28     | 36.32     |


/home/ubuntu/micromamba/envs/moml-ilya/lib/python3.11/site-packages/sklearn/metrics/_scorer.py:610: FutureWarning: The `needs_threshold` and `needs_proba` parameter are deprecated in version 1.4 and will be removed in 1.6. You can either let `response_method` be `None` or set it to `predict` to preserve the same behaviour.
  warnings.warn(
/home/ubuntu/micromamba/envs/moml-ilya/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/home/ubuntu/micromamba/envs/moml-ilya/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__

| 19        | 0.9989    | 0.2639    | 90.79     | 0.003398  | 75.62     | 45.5      |


/home/ubuntu/micromamba/envs/moml-ilya/lib/python3.11/site-packages/sklearn/metrics/_scorer.py:610: FutureWarning: The `needs_threshold` and `needs_proba` parameter are deprecated in version 1.4 and will be removed in 1.6. You can either let `response_method` be `None` or set it to `predict` to preserve the same behaviour.
  warnings.warn(
/home/ubuntu/micromamba/envs/moml-ilya/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/home/ubuntu/micromamba/envs/moml-ilya/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__

| 20        | 0.9978    | 0.2363    | 84.76     | 0.003042  | 73.91     | 43.83     |
Best parameters:  {'dropout_rate': 0.006139492719022066, 'input_dim': 94.08157339950867, 'learning_rate': 0.00793215140153552, 'num_neurons_layer1': 69.90478437564767, 'num_neurons_layer2': 31.874906370880705}


In [13]:
def create_model(input_dim, learning_rate, dropout_rate, num_neurons_layer1, num_neurons_layer2):
    model = Sequential()
    model.add(Dense(int(num_neurons_layer1), input_dim=input_dim, activation='relu', kernel_regularizer='l2'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(int(num_neurons_layer2), activation='relu', kernel_regularizer='l2'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(3, activation='sigmoid'))  # 3 output classes
    model.compile(loss='bce', optimizer=Adam(learning_rate=learning_rate), metrics=['accuracy'])
    return model

dnn = KerasClassifier(model=create_model, 
                      input_dim = int(best_params['input_dim']),
                      learning_rate=best_params['learning_rate'], 
                      dropout_rate=best_params['dropout_rate'], 
                      num_neurons_layer1=best_params['num_neurons_layer1'], 
                      num_neurons_layer2=best_params['num_neurons_layer2'],
                      epochs=50, 
                      batch_size=16, 
                      verbose=0)

pca = PCA(n_components = int(best_params['input_dim']))

X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(test.X)

StandardScaler()

dnn.fit(X_train_pca, y_train)

y_pred = dnn.predict(X_test_pca)
#print(y_pred)

y_prob = dnn.predict_proba(X_test_pca)
#y_prob = np.stack(y_prob, axis=1)
#print(y_prob)

y_pred_multi = {k: y_pred[:, idx] for idx, k in enumerate(benchmark.target_cols)}
y_prob_multi = {k: y_prob[:, idx] for idx, k in enumerate(benchmark.target_cols)}

results_multi = benchmark.evaluate(y_pred=y_pred_multi, y_prob=y_prob_multi)


results_multi.name = "dnn_multi_augm_pca"
results_multi.description = best_params
results_multi.to_json('dnn_multi_augm_pca.json')




/home/ubuntu/micromamba/envs/moml-ilya/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/home/ubuntu/micromamba/envs/moml-ilya/lib/python3.11/site-packages/pydantic/main.py:347: UserWarning: Pydantic serializer warnings:
  Expected `str` but got `dict` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


In [14]:
results_multi

{
  "name": "dnn_multi_augm_pca",
  "description": {
    "dropout_rate": 0.006139492719022066,
    "input_dim": 94.08157339950867,
    "learning_rate": 0.00793215140153552,
    "num_neurons_layer1": 69.90478437564767,
    "num_neurons_layer2": 31.874906370880705
  },
  "tags": [],
  "user_attributes": {},
  "owner": null,
  "polaris_version": "dev",
  "benchmark_name": "pkis1-kit-wt-mut-c-1",
  "benchmark_owner": {
    "slug": "polaris",
    "external_id": "org_2gtoaJIVrgRqiIR8Qm5BnpFCbxu",
    "type": "organization"
  },
  "github_url": null,
  "paper_url": null,
  "contributors": null,
  "artifact_id": null,
  "benchmark_artifact_id": "polaris/pkis1-kit-wt-mut-c-1",
  "results": [
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(T6701_mutant)",
      "Metric": "accuracy",
      "Score": 0.8390804598
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(V560G_mutant)",
      "Metric": "accuracy",
      "Score": 0.8620689655
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT",
      "Metric": "accuracy",
      "Score": 0.6206896552
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(T6701_mutant)",
      "Metric": "f1",
      "Score": 0.0
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(V560G_mutant)",
      "Metric": "f1",
      "Score": 0.0
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT",
      "Metric": "f1",
      "Score": 0.0
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(T6701_mutant)",
      "Metric": "roc_auc",
      "Score": 0.614481409
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(V560G_mutant)",
      "Metric": "roc_auc",
      "Score": 0.5211111111
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT",
      "Metric": "roc_auc",
      "Score": 0.6094276094
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(T6701_mutant)",
      "Metric": "pr_auc",
      "Score": 0.2544169887
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(V560G_mutant)",
      "Metric": "pr_auc",
      "Score": 0.194748974
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT",
      "Metric": "pr_auc",
      "Score": 0.4527244967
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(T6701_mutant)",
      "Metric": "mcc",
      "Score": 0.0
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(V560G_mutant)",
      "Metric": "mcc",
      "Score": 0.0
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT",
      "Metric": "mcc",
      "Score": 0.0
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(T6701_mutant)",
      "Metric": "cohen_kappa",
      "Score": 0.0
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(V560G_mutant)",
      "Metric": "cohen_kappa",
      "Score": 0.0
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT",
      "Metric": "cohen_kappa",
      "Score": 0.0
    }
  ]
}

In [60]:
best_params =  {'dropout_rate': 0.0013434256997184812, 'learning_rate': 0.0004996258264765169, 'num_neurons_layer1': 51.9807344803538, 'num_neurons_layer2': 41.68970050564931}

In [71]:
X_train, y_resampled = SMOTE(k_neighbors=2).fit_resample(X, ys_scalarized)
y_train = [inv_mapping[item] for i, item in enumerate(y_resampled)]
y_train = np.array(y_train)

def create_model(input_dim, learning_rate, dropout_rate, num_neurons_layer1, num_neurons_layer2):
    model = Sequential()
    model.add(Dense(int(num_neurons_layer1), input_dim=input_dim, activation='relu', kernel_regularizer='l2'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(int(num_neurons_layer2), activation='relu', kernel_regularizer='l2'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(3, activation='sigmoid'))  # 3 output classes
    model.compile(loss='bce', optimizer=Adam(learning_rate=learning_rate), metrics=['accuracy'])
    return model


input_dim = 100
dnn = KerasClassifier(model=create_model, 
                      input_dim = input_dim,
                      learning_rate=best_params['learning_rate'], 
                      dropout_rate=best_params['dropout_rate'], 
                      num_neurons_layer1=best_params['num_neurons_layer1'], 
                      num_neurons_layer2=best_params['num_neurons_layer2'],
                      epochs=50, 
                      batch_size=16, 
                      verbose=0)

pca = PCA(n_components = input_dim)

pipeline = Pipeline([
            ('pca', pca),
            ('scaler', StandardScaler()),
            ('dnn', dnn)
        ])
# Use K-Fold cross-validation
kfold = KFold(n_splits=2, shuffle=True, random_state=42)

# Evaluate the model using cross-validation
cv_scores = cross_val_score(pipeline, X_train, y_train, cv=kfold, scoring='accuracy')
print(cv_scores.mean())
#dnn.fit(X_train, y_train)

/home/ubuntu/micromamba/envs/moml-ilya/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/home/ubuntu/micromamba/envs/moml-ilya/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


0.9715447154471545


In [72]:
best_pipeline = pipeline

/home/ubuntu/micromamba/envs/moml-ilya/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/home/ubuntu/micromamba/envs/moml-ilya/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


ValueError: 
All the 2 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "/home/ubuntu/micromamba/envs/moml-ilya/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/ubuntu/micromamba/envs/moml-ilya/lib/python3.11/site-packages/sklearn/base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ubuntu/micromamba/envs/moml-ilya/lib/python3.11/site-packages/sklearn/pipeline.py", line 476, in fit
    self._final_estimator.fit(Xt, y, **last_step_params["fit"])
  File "/home/ubuntu/micromamba/envs/moml-ilya/lib/python3.